# Fine Tuning Resnet
- Create dataset class and instatiate dataloaders for batching large dataset on Kaggle
- Load in ResNet-18 weights from PyTorch library
- Allow for hyperparameter experimentation, including freezing/unfreezing layers.
- Plots training/validation loss curves
- Output log files, plot, and model weights for deployment

In [ ]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
from dotenv import dotenv_values 
import pickle as pkl

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

from scipy.spatial.distance import euclidean
from sklearn.model_selection import train_test_split
from transformers import WhisperProcessor, WhisperForConditionalGeneration

#### Create Dataset Class

In [ ]:
class SpectrogramDataset(Dataset):
    def __init__(self, file_paths, transform=False, sr=22050, n_mels=128):
        self.file_paths = file_paths
        self.data_index = self._build_index()
        self.sr = sr
        self.n_mels = n_mels
        self.transform = transform
        
    def _build_index(self):
        index = []
        for file_idx, file_path in enumerate(self.file_paths):
            with open(file_path, 'rb') as f:
                data = pkl.load(f)
                for i in range(len(data)):
                    index.append((file_idx, i))
        return index

    def _get_log_mel_spectrogram(self, y):
        # Convert to mel spectrogram
        mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=self.sr, n_mels=self.n_mels)
        # Convert to log scale
        log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
        log_mel_spectrogram = torch.tensor(log_mel_spectrogram, dtype=torch.float32).unsqueeze(0)
        return log_mel_spectrogram

    def __len__(self):
        return len(self.data_index)

    def __getitem__(self, idx):
        file_idx, data_idx = self.data_index[idx]
        file_path = self.file_paths[file_idx]

        with open(file_path, 'rb') as f:
            data = pkl.load(f)
        
        row = data.iloc[data_idx]
        anchor = row['processed_audio'][0]  # (y, sr)
        positive = row['augmented_audio'][0]
        negative = row['diff_processed_audio'][0]

        # Convert to log mel spectrograms
        anchor_mel = self._get_log_mel_spectrogram(anchor)
        positive_mel = self._get_log_mel_spectrogram(positive)
        negative_mel = self._get_log_mel_spectrogram(negative)
        
        # Apply any transformations
        if self.transform:
            anchors = self.transform(anchors)
            positives = self.transform(positives)
            negatives = self.transform(negatives)

        return anchor_mel, positive_mel, negative_mel


### Split Data and Instantiate Dataset Class

In [ ]:
file_paths = [f'/kaggle/input/augmented-audio-10k/batch_{i}_augmented.pkl' for i in range(1,10,1)]

# Split the files instaed of actual data into training/val
train_files, val_files = train_test_split(file_paths, test_size=0.2, random_state=123)

# Instantiate Dataset Classes
train_dataset = SpectrogramDataset(train_files)
val_dataset = SpectrogramDataset(val_files)

# Declare dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)#, drop_last=True)

### Delcaring the Model
- Define architecture: default resnet with adjusted first conv layer and final FC layer to set num params
- Choose loss function, optimizer, device, etc.

In [ ]:
# Define pretrained resnet from Torch Vision resnet 18
class ResNetEmbedding(nn.Module):
    def __init__(self, embedding_dim=128, dropout_rate=0.5):
        # get resnet super class
        super(ResNetEmbedding, self).__init__()
        self.resnet = models.resnet18(weights='DEFAULT')
        # Change structure of first layer to take non RGB images, rest of params same as default
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.dropout = nn.Dropout(p=dropout_rate)
        # Set the last fully connected to a set dimension "embedding_dim" instead of default 1000
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, embedding_dim)

    def forward(self, x):
        x = self.resnet(x)
        return F.normalize(x, p=2, dim=1)

In [ ]:
# Choose model, loss, and optimizer
model = ResNetEmbedding()

criterion = nn.TripletMarginLoss(margin=1.0, p=2, eps=1e-7)

# --- COMMENT OUT BELOW IF NOT FREEZING ----- #
# Freeze all the layers
for param in model.resnet.parameters():
    param.requires_grad = False

# Turn back on last residual block
for param in model.resnet.layer4.parameters():
    param.requires_grad = True

# Turn back on fully connected layer
for param in model.resnet.fc.parameters():
    param.requires_grad = True

# Use a smaller learning rate for fine-tuning
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
# --- COMMENT OUT ABOVE IF NOT FREEZING --- #

# Uncomment if not freezing
#optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-4)

# Declare losses/accuracies
train_losses = []
val_losses = []
baseline_losses = []

num_epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

### Training Model

In [ ]:
# Loop over epochs
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    running_train_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Training {epoch+1}/{num_epochs}", unit="batch")

    # Loop over batches using dataloaders
    for anchors, positives, negatives in train_loader:
        anchors, positives, negatives = anchors.to(device), positives.to(device), negatives.to(device)
        optimizer.zero_grad()
        
        anchor_embeddings = model(anchors)
        positive_embeddings = model(positives)
        negative_embeddings = model(negatives)
        
        loss = criterion(anchor_embeddings, positive_embeddings, negative_embeddings)
        loss.backward()
        optimizer.step()
        
        running_train_loss += loss.item() * anchors.size(0)
        pbar.update(1)

          
    train_loss = running_train_loss / len(train_loader.dataset)
    train_losses.append(train_loss)

    # Turn on validation/eval mode
    model.eval()
    running_val_loss = 0.0 
    running_baseline_loss = 0.0   
    val_pbar = tqdm(val_loader, desc=f"Validation {epoch+1}/{num_epochs}", unit="batch")
    # Turn off gradient updates since we're in validation
    with torch.no_grad():
        # Batch loop 
        for anchors, positives, negatives in tqdm(val_loader):
            anchors, positives, negatives = anchors.to(device), positives.to(device), negatives.to(device)
            
            anchor_embeddings = model(anchors)
            positive_embeddings = model(positives)
            negative_embeddings = model(negatives)
            
            loss = criterion(anchor_embeddings, positive_embeddings, negative_embeddings)

            # Add to running val loss
            running_val_loss += loss.item() * anchors.size(0)
            
            # baseline loss
            baseline_loss = criterion(F.normalize(anchors), 
                                               F.normalize(positives), 
                                               F.normalize(negatives)).item()
            running_baseline_loss += baseline_loss*anchors.size(0)
            val_pbar.update(1) 

    
    # Calculate average validation loss over the entire dataset
    val_loss = running_val_loss / len(val_loader.dataset)
    val_losses.append(val_loss)
    # Do the same for the baseline
    baseline_avg_loss = running_baseline_loss / len(val_loader.dataset)
    baseline_losses.append(baseline_avg_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Baseline Loss: {baseline_avg_loss:.4f}")
    
    with open('training_logs.pkl', 'wb') as f:
        pkl.dump((train_losses, val_losses, baseline_losses), f)

In [ ]:
# Plot loss curves for training
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(10, 5))
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.plot(epochs, baseline_losses, label='Baseline Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.tight_layout()
plt.savefig('resnet-loss-plot.png')
plt.show()


In [ ]:
# Save just the model weights (recommended apparently for portability/compatibility)
torch.save(model.state_dict(), 'resnet18_model_weights.pth')

In [ ]:
# Save the entire model so we can use it for deployment
torch.save(model, 'resnet18_model.pth')